In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
import time
from datetime import datetime

In [7]:
#seasons = pd.read_csv('data/j_seasons.csv')
episodes = pd.read_csv('data/j_episodes.csv')

In [8]:
episodes

,episode_url
0,http://www.j-archive.com/showgame.php?game_id=...
1,http://www.j-archive.com/showgame.php?game_id=...
2,http://www.j-archive.com/showgame.php?game_id=...
3,http://www.j-archive.com/showgame.php?game_id=...
4,http://www.j-archive.com/showgame.php?game_id=...
...,...
72,http://www.j-archive.com/showgame.php?game_id=...
73,http://www.j-archive.com/showgame.php?game_id=...
74,http://www.j-archive.com/showgame.php?game_id=...
75,http://www.j-archive.com/showgame.php?game_id=...


In [9]:
episodes = episodes[0:10]
episodes

,episode_url
0,http://www.j-archive.com/showgame.php?game_id=...
1,http://www.j-archive.com/showgame.php?game_id=...
2,http://www.j-archive.com/showgame.php?game_id=...
3,http://www.j-archive.com/showgame.php?game_id=...
4,http://www.j-archive.com/showgame.php?game_id=...
5,http://www.j-archive.com/showgame.php?game_id=...
6,http://www.j-archive.com/showgame.php?game_id=...
7,http://www.j-archive.com/showgame.php?game_id=...
8,http://www.j-archive.com/showgame.php?game_id=...
9,http://www.j-archive.com/showgame.php?game_id=...


In [10]:
for index, row in episodes.iterrows():
    url = row['episode_url']
    print(url)

http://www.j-archive.com/showgame.php?game_id=6902
http://www.j-archive.com/showgame.php?game_id=6901
http://www.j-archive.com/showgame.php?game_id=6900
http://www.j-archive.com/showgame.php?game_id=6899
http://www.j-archive.com/showgame.php?game_id=6898
http://www.j-archive.com/showgame.php?game_id=6897
http://www.j-archive.com/showgame.php?game_id=6896
http://www.j-archive.com/showgame.php?game_id=6895
http://www.j-archive.com/showgame.php?game_id=6894
http://www.j-archive.com/showgame.php?game_id=6893


In [11]:
for index, row in episodes.iterrows(): 
    time.sleep(20)
    url = row['episode_url']
    print(url)
    response = requests.get(url)
    soup = BeautifulSoup(response.text)
    #get the show header info
    header = soup.find('h1')
    header = header.text
    show_num = header.split('-')[0].strip().split('#')[1]
    air_date = datetime.strftime(datetime.strptime(header.split('-')[1].strip().split(',',1)[1].strip(),'%B %d, %Y'), '%Y-%m-%d')
    ##print(show_num, ', ', air_date)
    
    #find the season
    season = soup.find('a', text='[current season]')
    ##print(season)
    season = season['href'].split('=')[1]

    #get the round scores
    scores = soup.findAll(['h3','table'])
    score_texts = np.array([['Scores at the end of the Jeopardy! Round:', 'J'],
                            ['Scores at the end of the Double Jeopardy! Round:', 'DJ'],
                            ['Final scores:', 'FJ']])
    j_round = []
    j_name = []
    j_score = []
    for x in score_texts:
        for element in scores:
            if element.text == x[0]:
                while element.name != 'table':
                    element = element.next_sibling
                    #print('name: ',element.name)
                score_table = element.findAll('tr')
                [j_round.append(x[1]) for td in score_table[0].findAll('td')]
                [j_name.append(td.text) for td in score_table[0].findAll('td')]
                [j_score.append(td.text) for td in score_table[1].findAll('td')]

    j_scores = pd.DataFrame({'show_num': show_num, 'round_code': j_round, 'contestant': j_name, 'score': j_score})

    #get the game stats
    remarks_text = 'Coryat scores:'
    j_right = []
    j_ddright = []
    j_wrong = []
    j_ddwrong = []
    for element in scores:
        if element.text == remarks_text:
            while element.name != 'table':
                element = element.next_sibling
            remarks_table = element.findAll('td', class_ = 'score_remarks')
            [j_right.append((re.findall('(\d{1,2}) R', x.text.split(',')[0]) or list('0'))[0]) for x in remarks_table]
            [j_ddright.append((re.findall('(\d{1}) DD', x.text.split(',')[0]) or list('0'))[0]) for x in remarks_table]
            [j_wrong.append((re.findall('(\d{1,2}) W', x.text.split(',')[1]) or list('0'))[0]) for x in remarks_table]
            [j_ddwrong.append((re.findall('(\d{1}) DD', x.text.split(',')[1]) or list('0'))[0]) for x in remarks_table]

    j_remarks = pd.DataFrame({'contestant': j_name[:3], 'num_right': j_right, 'num_ddright': j_ddright, 'num_wrong': j_wrong,
                             'num_ddwrong': j_ddwrong})
    ##j_remarks
    
    #merge
    j_scores = j_scores.merge(j_remarks, how = 'left', on='contestant')
    
    #get the finish order
    finish_order = []
    winner = j_scores[j_scores.round_code=='FJ'].sort_values(['score'], ascending = False)['contestant'].iloc[0]
    second = j_scores[j_scores.round_code=='FJ'].sort_values(['score'], ascending = False)['contestant'].iloc[1]
    third = j_scores[j_scores.round_code=='FJ'].sort_values(['score'], ascending = False)['contestant'].iloc[2]
    finish_order.append([winner, 'Winner'])
    finish_order.append([second, 'Second'])
    finish_order.append([third, 'Third'])
    finish_order = pd.DataFrame(finish_order)
    finish_order.columns=(['contestant', 'finish'])
    
    #merge
    j_scores = j_scores.merge(finish_order, how = 'left', on = 'contestant')
    #append to the dataset
    j_scores.to_csv('data/j_scores.csv', mode = 'a', index=False, header=False)
    
    #build the categories df
    categories = soup.findAll('td', attrs = {'class': 'category_name'})
    categories = np.array([x.text for x in categories])
    rounds = []
    [[rounds.append(item) for x in range(6)] for item in ['J', 'DJ']]
    rounds.append('FJ')
    rounds = np.array(rounds)
    catnums = np.array(['1', '2', '3', '4', '5', '6', '1', '2', '3', '4', '5', '6', '1'])
    categories = pd.DataFrame({'category': categories, 'round_code': rounds, 'cat_num': catnums})

    #add the header data collected previously
    categories['season'] = season
    categories['show_num'] = show_num
    categories['air_date'] = air_date
    ##categories
    
    #process all the clues except FJ
    clues = soup.find_all('td', attrs = {'class': 'clue'})
    all_rows = []
    for clue in clues:
        ##print(type(clue))
        new_row = []
        #Get our clue number, which tells us the round, category, and board position
        clue_id = clue.find('td', class_='clue_text')
        if not clue_id == None:
            ##print(clue_id)
            clue_id = clue.find('td', class_='clue_text')['id']
            #What round is this?
            round_code = clue_id.split('_')[1]
            cat_num = clue_id.split('_')[2] if not round_code == 'FJ' else 'checkme'
            clue_num = clue_id.split('_')[3] if not round_code == 'FJ' else 'checkme'
            if not round_code == 'FJ':
                #Get our clue value
                #DD does not have a clue_value, it has a clue_value_daily_double
                #For DD split out the dollar value and set the flag
                clue_value = clue.find('td', class_='clue_value_daily_double').text.split(' ')[1] if not clue.find('td', {'class':'clue_value'}) else clue.find('td', {'class':'clue_value'}).text
                daily_double = False if not clue.find('td', class_='clue_value_daily_double') else True
                clue_order = clue.find('td', class_='clue_order_number').text
                clue_text = clue.find('td', class_='clue_text').text
                corr_resp = BeautifulSoup(clue.find('div', onmouseover=True).get('onmouseover'))
                ##print(corr_resp)
                #Check for clues with no correct responder, and set the flag
                corr_respr1 = '' if not corr_resp.find('td', class_='right') else corr_resp.find('td', class_='right').text
                corr_FJ_wager1 = ''
                corr_respr2 = ''
                corr_FJ_wager2 = ''
                corr_respr3 = ''
                corr_FJ_wager3 = ''
                triple_stumper = True if not corr_resp.find('td', class_='right') else False
                incorr_resprs = corr_resp.findAll('td', class_='wrong')
                #Okay, THIS is ugly
                incorr_respr1 = ''
                incorr_FJ_wager1 = ''
                incorr_respr2 = ''
                incorr_FJ_wager2 = ''
                incorr_respr3 = ''
                incorr_FJ_wager3 = ''
                if len(incorr_resprs) >= 1:
                    incorr_respr1 = '' if incorr_resprs[0].text == 'Triple Stumper' else incorr_resprs[0].text
                if len(incorr_resprs) >= 2:
                    incorr_respr2 = '' if incorr_resprs[1].text == 'Triple Stumper' else incorr_resprs[1].text
                if len(incorr_resprs) == 3:
                    incorr_respr3 = '' if incorr_resprs[2].text == 'Triple Stumper' else incorr_resprs[2].text
                corr_resp = corr_resp.find('em', class_='correct_response').text
                new_row = [round_code, cat_num, clue_num, clue_id, clue_value, daily_double, clue_order, clue_text, corr_resp, 
                           corr_respr1, corr_FJ_wager1, corr_respr2, corr_FJ_wager2, corr_respr3, corr_FJ_wager3, 
                           triple_stumper, incorr_respr1, incorr_FJ_wager1, incorr_respr2, incorr_FJ_wager2, incorr_respr3, incorr_FJ_wager3 ]
                all_rows.append(new_row)
                ##all_rows

    #process the FJ clue
    round_code = 'FJ'
    cat_num = '1'
    clue_num = '1'
    clue_id = 'clue_FJ'
    clue_value = ''
    daily_double = False
    clue_order = ''
    clue_text = ''
    corr_resp = ''
    triple_stumper = True

    final_round = soup.find('table', class_ = 'final_round').find_all('td')
    ##type(final_round)
    #final_round - this will absolutely barf and so will a print; it gives a type error

    for item in final_round:
        #this only works because it's the first item! aaargh
        stats = BeautifulSoup(item.find('div', onmouseover=True).get('onmouseover'))
        if stats:
            break
    corr_resp = stats.find('em')
    corr_resp = corr_resp.get_text()

    #make a dictionary of responders
    resprs = {}
    wagers = stats.findAll('td')

    right = 0
    wrong = 0
    for td in wagers:
        td_text = td.get_text()
        if td.has_attr('class'):
            td_class = td.get('class')[0]
            if td_class == 'right':
                triple_stumper = False
                right +=1
                key_name = 'corr_respr' + str(right)
                resprs[key_name] = td_text
                wager_key = 'corr_FJ_wager' + str(right)
            else:
                wrong +=1
                key_name = 'incorr_respr' + str(wrong)
                resprs[key_name] = td_text
                wager_key = 'incorr_FJ_wager' + str(wrong)
        if td.has_attr('rowspan'):
            continue
        if not td.has_attr('class'):
            resprs[wager_key] = td_text

    corr_respr1 = resprs.get('corr_respr1', '')
    corr_FJ_wager1 = resprs.get('corr_FJ_wager1', '')
    corr_respr2 = resprs.get('corr_respr2', '')
    corr_FJ_wager2 = resprs.get('corr_FJ_wager2', '')
    corr_respr3 = resprs.get('corr_respr3', '')
    corr_FJ_wager3 = resprs.get('corr_FJ_wager3', '')
    incorr_respr1 = resprs.get('incorr_respr1','')
    incorr_FJ_wager1 = resprs.get('incorr_FJ_wager1', '')
    incorr_respr2 = resprs.get('incorr_respr2','')
    incorr_FJ_wager2 = resprs.get('incorr_FJ_wager2', '')
    incorr_respr3 = resprs.get('incorr_respr3','')
    incorr_FJ_wager3 = resprs.get('incorr_FJ_wager3', '')

    clue_text = soup.find('td', attrs = {'class': 'clue_text', 'id': 'clue_FJ'})
    clue_text = clue_text.get_text()
    ##print(clue_text)

    new_row = [round_code, cat_num, clue_num, clue_id, clue_value, daily_double, clue_order, clue_text, corr_resp, 
                       corr_respr1, corr_FJ_wager1, corr_respr2, corr_FJ_wager2, corr_respr3, corr_FJ_wager3, 
                       triple_stumper, incorr_respr1, incorr_FJ_wager1, incorr_respr2, incorr_FJ_wager2, incorr_respr3, incorr_FJ_wager3]
    all_rows.append(new_row)
    #make df of all clues for this show
    this_show = pd.DataFrame(all_rows)
    this_show.columns = ['round_code', 'cat_num', 'clue_num', 'clue_id', 'clue_value', 'daily_double', 'clue_order', 'clue_text', 'corr_resp', 
                       'corr_respr1', 'corr_FJ_wager1', 'corr_respr2', 'corr_FJ_wager2', 'corr_respr3', 'corr_FJ_wager3', 
                       'triple_stumper', 'incorr_respr1', 'incorr_FJ_wager1', 'incorr_respr2', 'incorr_FJ_wager2', 'incorr_respr3', 'incorr_FJ_wager3']
    this_show = this_show.merge(categories, how='left', left_on=['round_code', 'cat_num'], right_on=['round_code', 'cat_num'])
    ##this_show
    #append to the dataset
    this_show.to_csv('data/j_clues.csv', mode='a', index=False, header=False)

http://www.j-archive.com/showgame.php?game_id=6902
http://www.j-archive.com/showgame.php?game_id=6901
http://www.j-archive.com/showgame.php?game_id=6900
http://www.j-archive.com/showgame.php?game_id=6899
http://www.j-archive.com/showgame.php?game_id=6898
http://www.j-archive.com/showgame.php?game_id=6897
http://www.j-archive.com/showgame.php?game_id=6896
http://www.j-archive.com/showgame.php?game_id=6895
http://www.j-archive.com/showgame.php?game_id=6894
http://www.j-archive.com/showgame.php?game_id=6893
